In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set the path to dataset
import os
home = '/content/drive/MyDrive/2ReadyForFineTuning/VLQAv1/Charts/'
imroot = home+'images'
os.chdir(home)

In [ ]:
# Prepare dataset in a desired format
prefix = 'charts'

import json
import ast
datalen = 0
dumpdict = { "info": {}, "task_type": "Open-Ended", "data_type": "clevr", "license": {}, "data_subtype": prefix, "questions": [] }

with open(home+prefix+'.jsonl') as f:
  data = f.readlines()
  print(len(data))
  global datalen
  datalen = len(data)
  for i in data:
    ijson = json.loads(i)
    imlist = ast.literal_eval(ijson['images'])
    #print(imlist)
    anslist = ast.literal_eval(ijson['answer_choices'])
    if len(anslist)==2:
      anslist.append("Unknown")
      anslist.append("Unanswerable")
    try:
      #print(len(anslist))
      assert len(anslist) == 4
    except:
      print(i)
    dumpdict['questions'].append( { "image_id": imroot+"/"+imlist[0].replace("./images/", ""), "question": ijson['question']+" "+ijson['passage'], "question_id": ijson['qid'], "answer_choices": anslist, "answer_id": int(ijson['answer']) } )
with open('/content/'+prefix+'.jsonl','w+') as w:
  w.write(json.dumps(dumpdict,indent=4))

print(dumpdict)

251
{'info': {}, 'task_type': 'Open-Ended', 'data_type': 'clevr', 'license': {}, 'data_subtype': 'charts', 'questions': [{'image_id': '/content/drive/MyDrive/2ReadyForFineTuning/VLQAv1/Charts/images/21.png', 'question': 'From the above graph infer comparatively which country is most likely to have highest per capita income? The above visualization shows health care expenditure across countries. There is a strong correlation between health care expenditure and national income. The countries which spend more on health care are more likely to have highest per capita income.', 'question_id': 2218, 'answer_choices': ['United States', 'United Kingdom', 'India', 'Iraq'], 'answer_id': 0}, {'image_id': '/content/drive/MyDrive/2ReadyForFineTuning/VLQAv1/Charts/images/21.png', 'question': 'How many countries are most likely to have better per capita income level than Iraq? The above visualization shows health care expenditure across countries. There is a strong correlation between health care exp

In [ ]:
# Load prepared dataset
import json
f = open('/content/'+prefix+'.jsonl','r')
data_questions = json.load(f)
questions = data_questions['questions']

print(data_questions.keys())
print("Number of questions:", len(questions))
print(questions[0])

dict_keys(['info', 'task_type', 'data_type', 'license', 'data_subtype', 'questions'])
Number of questions: 251
{'image_id': '/content/drive/MyDrive/2ReadyForFineTuning/VLQAv1/Charts/images/21.png', 'question': 'From the above graph infer comparatively which country is most likely to have highest per capita income? The above visualization shows health care expenditure across countries. There is a strong correlation between health care expenditure and national income. The countries which spend more on health care are more likely to have highest per capita income.', 'question_id': 2218, 'answer_choices': ['United States', 'United Kingdom', 'India', 'Iraq'], 'answer_id': 0}


In [ ]:
# Install necessary libraries
!pip install torch
!pip install salesforce-lavis
!pip install transformers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 16.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 5.1 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 66.1 MB/s eta 0

In [6]:
# Import necessary libraries
import torch
from PIL import Image
from lavis.models import load_model_and_preprocess
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from transformers import AutoProcessor, AutoModelForCausalLM
from huggingface_hub import hf_hub_download

/usr/local/lib/python3.10/dist-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/usr/local/lib/python3.10/dist-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


In [7]:
# Check if GPU is available (recommended but not necessary)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
# Load BLIP model finetuned on VQAv2
model_blip_vqa2, vis_processors_blip_vqa2, txt_processors_blip_vqa2 = load_model_and_preprocess(name="blip_vqa", model_type="vqav2", is_eval=True, device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

100%|██████████| 1.35G/1.35G [01:15<00:00, 19.1MB/s]
/usr/local/lib/python3.10/dist-packages/lavis/models/base_model.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  che

# Batch Run

In [9]:
# Create a batch run script that executes BLIP on an item from the dataset
def batchrun(impath, question, answers):
  raw_image = Image.open(impath).convert("RGB")

  image_blip_vqa2 = vis_processors_blip_vqa2["eval"](raw_image).unsqueeze(0).to(device)
  question_blip_vqa2 = txt_processors_blip_vqa2["eval"](question)
  samples_blip_vqa2 = {"image": image_blip_vqa2, "text_input": question_blip_vqa2}
  bvqa2 = model_blip_vqa2.predict_answers(samples_blip_vqa2, answer_list=answers, inference_method="rank")[0]

  return bvqa2


In [13]:
# Call batch run script and write results as csv file
import csv
filprefix = "IPQ_BLIP_"+prefix.upper()

with open(filprefix+'.csv', 'w+') as k:
  spamwriter = csv.writer(k)
  spamwriter.writerow(["qid","gt_ans","pred_ans_blip","correctness_blip"])
  for exid in range(0,datalen):
    print(exid)
    correctness_blip = 0
    print(questions[exid]['image_id'], questions[exid]['question'], questions[exid]['answer_choices'])
    pred_ans_blip = batchrun(questions[exid]['image_id'], questions[exid]['question'], [str(i) for i in questions[exid]['answer_choices']])
    gtclass = questions[exid]['answer_choices'][int(questions[exid]['answer_id'])]
    if str(gtclass)==str(pred_ans_blip):
      correctness_blip = 1
    print(exid,gtclass,pred_ans_blip,correctness_blip)
    spamwriter.writerow([exid,gtclass,pred_ans_blip,correctness_blip])

0
/content/drive/MyDrive/2ReadyForFineTuning/VLQAv1/Charts/images/21.png From the above graph infer comparatively which country is most likely to have highest per capita income? The above visualization shows health care expenditure across countries. There is a strong correlation between health care expenditure and national income. The countries which spend more on health care are more likely to have highest per capita income. ['United States', 'United Kingdom', 'India', 'Iraq']
0 United States United Kingdom 0
1
/content/drive/MyDrive/2ReadyForFineTuning/VLQAv1/Charts/images/21.png How many countries are most likely to have better per capita income level than Iraq? The above visualization shows health care expenditure across countries. There is a strong correlation between health care expenditure and national income. The countries which spend more on health care are more likely to have highest per capita income. [2, 3, 4, 1]
1 2 1 0
2
/content/drive/MyDrive/2ReadyForFineTuning/VLQAv1/C